## Import Libraries


In [81]:
import pandas as pd

import statsmodels.api as sm

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

import joblib
import requests,json
from joblib import dump, load

In [82]:
df = pd.read_csv("MLCC.csv") # Load File
df.shape

(283065, 59)

In [83]:
df.head()

ClaimNumber   EvalResult  6054  6055  6056  6057  6058  6060  6061  6062  \
0            1            0     0     0     0     0     0     0     0     0   
1            2            0     0     0     0     0     0     0     0     0   
2            3            0     0     0     0     0     0     0     0     1   
3            4            0     0     0     0     0     0     0     0     0   
4            5            0     0     0     0     0     0     0     0     0   

   ...  7928  7929  7930  7931  8445  8446  8447  8448  8449  8450  
0  ...     0     0     0     0     0     0     0     0     0     0  
1  ...     0     0     0     0     0     1     1     1     1     0  
2  ...     0     0     0     0     1     2     2     2     1     0  
3  ...     0     0     0     0     1     3     2     3     1     0  
4  ...     0     0     0     0     1     4     2     4     1     0  

[5 rows x 59 columns]

## Cleaning the data (removing noice)

In [84]:
df.iloc[:,1].value_counts()  # for EvalRes

0    238322
1     44742
2         1
Name:  EvalResult, dtype: int64

In [85]:
df = df[df.iloc[:,1] != 2]  # Removing noise 
df = df.dropna()
df.shape

(283064, 59)

In [86]:
df.iloc[:,1].value_counts() # For EvalRes

0    238322
1     44742
Name:  EvalResult, dtype: int64

In [87]:
df.describe()

ClaimNumber     EvalResult          6054           6055  \
count  283064.000000  283064.000000  283064.00000  283064.000000   
mean   141533.374541       0.158063       0.01683       0.010559   
std     81713.872676       0.364801       0.18456       0.138878   
min         1.000000       0.000000       0.00000       0.000000   
25%     70767.750000       0.000000       0.00000       0.000000   
50%    141533.500000       0.000000       0.00000       0.000000   
75%    212299.250000       0.000000       0.00000       0.000000   
max    283065.000000       1.000000       3.00000       3.000000   

                6056           6057           6058           6060  \
count  283064.000000  283064.000000  283064.000000  283064.000000   
mean        0.025083       0.038942       0.039072       0.080756   
std         0.214089       0.264971       0.301434       0.511833   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         0.000000       0.000000       0.000000       0.000000   
max         3.000000       3.000000       3.000000       6.000000   

                6061           6062  ...           7928           7929  \
count  283064.000000  283064.000000  ...  283064.000000  283064.000000   
mean        0.113865       0.295064  ...       0.237914       0.474377   
std         0.590368       0.917938  ...       1.520425       2.970075   
min         0.000000       0.000000  ...       0.000000       0.000000   
25%         0.000000       0.000000  ...       0.000000       0.000000   
50%         0.000000       0.000000  ...       0.000000       0.000000   
75%         0.000000       0.000000  ...       0.000000       0.000000   
max         6.000000       8.000000  ...      37.000000      42.000000   

                7930           7931           8445           8446  \
count  283064.000000  283064.000000  283064.000000  283064.000000   
mean        0.001226       0.000728       0.611353       4.035402   
std         0.047867       0.082655       0.985680       3.833302   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       2.000000   
50%         0.000000       0.000000       0.000000       3.000000   
75%         0.000000       0.000000       1.000000       5.000000   
max         3.000000      16.000000       4.000000      52.000000   

                8447           8448           8449           8450  
count  283064.000000  283064.000000  283064.000000  283064.000000  
mean        1.162656       9.416994       0.979821       0.034688  
std         0.616582       9.001560       0.239840       0.823398  
min         0.000000       0.000000       0.000000       0.000000  
25%         1.000000       2.000000       1.000000       0.000000  
50%         1.000000       6.000000       1.000000       0.000000  
75%         1.000000      14.000000       1.000000       0.000000  
max         3.000000      42.000000       3.000000      40.000000  

[8 rows x 59 columns]

## Splitting the data

In [88]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
   df.iloc[:,2:], df.iloc[:,1], test_size=0.2, random_state=42)

## ML Models

### SVM Algorithm

In [89]:
%%time
# Training
clf_svm = SVC(gamma='auto')
clf_svm.fit(X_train, y_train)

CPU times: user 17min 51s, sys: 4.9 s, total: 17min 55s
Wall time: 18min 6s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [90]:
# Testing
y_pred = clf_svm.predict(X_test)   # Prediction using SVM

In [91]:
# Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97     47570
           1       0.89      0.80      0.84      9043

    accuracy                           0.95     56613
   macro avg       0.93      0.89      0.91     56613
weighted avg       0.95      0.95      0.95     56613



In [92]:
joblib.dump(clf_svm, 'model_svm.pkl')
print("SVM Model dumped!")

# Load the model that is just saved
clf_svm = joblib.load('model_svm.pkl')

SVM Model dumped!


### Trying Logistic Regression Algorithm

In [93]:
%%time
results = sm.OLS(y_train,X_train).fit()
results.summary() 

CPU times: user 5.54 s, sys: 1 s, total: 6.54 s
Wall time: 2.24 s


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             EvalResult   R-squared (uncentered):                   0.520
Model:                            OLS   Adj. R-squared (uncentered):              0.520
Method:                 Least Squares   F-statistic:                              4306.
Date:                Tue, 26 Jan 2021   Prob (F-statistic):                        0.00
Time:                        12:50:02   Log-Likelihood:                         -29002.
No. Observations:              226451   AIC:                                  5.812e+04
Df Residuals:                  226394   BIC:                                  5.871e+04
Df Model:                          57                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
6054           0.0018      0.003      0.561      0.575      -0.004       0.008
6055           0.0230      0.005      5.093      0.000       0.014       0.032
6056          -0.0064      0.003     -2.267      0.023      -0.012      -0.001
6057           0.0355      0.002     15.641      0.000       0.031       0.040
6058           0.0030      0.002      1.433      0.152      -0.001       0.007
6060           0.0078      0.001      6.256      0.000       0.005       0.010
6061           0.0097      0.001      9.412      0.000       0.008       0.012
6062           0.0064      0.001      8.637      0.000       0.005       0.008
6063          -0.0064      0.002     -2.852      0.004      -0.011      -0.002
6064           0.0086      0.001      9.117      0.000       0.007       0.010
6065          -0.0557      0.002    -27.247      0.000      -0.060      -0.052
6066           0.0354      0.004      9.678      0.000       0.028       0.043
6067           0.1231      0.010     12.537      0.000       0.104       0.142
6068           0.0289      0.007      3.892      0.000       0.014       0.043
6069          -0.0264      0.004     -6.938      0.000      -0.034      -0.019
6070           0.0416      0.002     18.662      0.000       0.037       0.046
6071          -0.0040      0.002     -2.092      0.036      -0.008      -0.000
6072           0.0295      0.003      9.879      0.000       0.024       0.035
6073           0.0355      0.003     12.735      0.000       0.030       0.041
6074          -0.0003      0.002     -0.109      0.913      -0.005       0.005
6075           0.0659      0.003     18.847      0.000       0.059       0.073
6076           0.0889      0.004     21.548      0.000       0.081       0.097
6077           0.0240      0.001     19.536      0.000       0.022       0.026
6078           0.1645      0.010     16.029      0.000       0.144       0.185
6079           0.2505      0.015     16.378      0.000       0.221       0.281
6080           0.0846      0.003     30.765      0.000       0.079       0.090
6081           0.0508      0.009      5.654      0.000       0.033       0.068
6082           0.0806      0.003     23.546      0.000       0.074       0.087
6083           0.0199      0.003      7.190      0.000       0.014       0.025
6084           0.1122      0.008     14.066      0.000       0.097       0.128
6085           0.0094      0.001     14.907      0.000       0.008       0.011
6087           0.0570      0.005     11.289      0.000       0.047       0.067
6088          -0.0044      0.001     -3.980      0.000      -0.007      -0.002
6089          -0.0012      0.008     -0.157      0.876      -0.016       0.014
7778           0.0126      0.001     15.936      0.000       

In [94]:
clf_LR = LogisticRegression(random_state=0).fit(X_train, y_train)

/Users/piyushkurkure/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [95]:
y_pred = clf_LR.predict(X_test)

In [96]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94     47570
           1       0.75      0.55      0.63      9043

    accuracy                           0.90     56613
   macro avg       0.84      0.76      0.79     56613
weighted avg       0.89      0.90      0.89     56613



In [97]:
joblib.dump(clf_LR, 'model_LogisticRegression.pkl')
print("Logistic Regression Model dumped!")

# Load the model that is just saved
clf_LR = joblib.load('model_LogisticRegression.pkl')

Logistic Regression Model dumped!


### Trying Random Forest Algorithm

In [98]:
%%time
clf_RF = RandomForestClassifier(max_depth=10, random_state=0)
clf_RF.fit(X_train, y_train)

/Users/piyushkurkure/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 1.5 s, sys: 80.2 ms, total: 1.58 s
Wall time: 1.32 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [99]:
y_pred = clf_RF.predict(X_test)
accuracy_score(y_test, y_pred)

0.9329305989790331

In [100]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96     47570
           1       0.94      0.62      0.75      9043

    accuracy                           0.93     56613
   macro avg       0.94      0.81      0.85     56613
weighted avg       0.93      0.93      0.93     56613



#### Selecting max dept

In [101]:
%%time
max_ = 0
for i in range(1,20):
    clf_RF_dept = RandomForestClassifier(max_depth=i, random_state=22 , n_estimators=50)
    clf_RF_dept.fit(X_train, y_train)
    y_pred = clf_RF_dept.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    if(acc > max_):
        max_ = acc
        print(max_,"DEPT =", i) 

0.8402663699150372 DEPT = 1
0.8774839701128716 DEPT = 3
0.8810343913942027 DEPT = 4
0.8912263967639942 DEPT = 5
0.9076713828979209 DEPT = 6
0.9130765018635296 DEPT = 7
0.9172628194937559 DEPT = 8
0.9274018335011394 DEPT = 9
0.9415681910515252 DEPT = 10
0.9419567943758501 DEPT = 12
0.9498878349495699 DEPT = 13
0.9534382562309011 DEPT = 14
0.9555755745146874 DEPT = 15
0.9616519174041298 DEPT = 16
0.9650080370232985 DEPT = 17
0.9659972091215798 DEPT = 18
0.9692826735908714 DEPT = 19
CPU times: user 1min 53s, sys: 2.5 s, total: 1min 56s
Wall time: 1min 57s


In [102]:
clf_RF_dept = RandomForestClassifier(max_depth=12, random_state=22 , n_estimators=50)
clf_RF_dept.fit(X_train, y_train) 
y_pred = clf_RF_dept.predict(X_test)

In [103]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97     47570
           1       0.94      0.68      0.79      9043

    accuracy                           0.94     56613
   macro avg       0.94      0.84      0.88     56613
weighted avg       0.94      0.94      0.94     56613



In [104]:
joblib.dump(clf_RF_dept, 'model_RandomForest.pkl')
print("Random Forest Model dumped!")

# Load the model that is just saved
clf_RF_dept = joblib.load('model_RandomForest.pkl')

Random Forest Model dumped!


### Using Pipelines

In [105]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
esti_svm = [('dim_reduction', PCA())
            , ('classifier', SVC())]
pipe = Pipeline(esti_svm)

In [106]:
%%time
pipe.set_params(dim_reduction__n_components=10, classifier__C=.1).fit(X_train, y_train)

/Users/piyushkurkure/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


CPU times: user 8min 18s, sys: 3.53 s, total: 8min 21s
Wall time: 8min 19s


Pipeline(memory=None,
         steps=[('dim_reduction',
                 PCA(copy=True, iterated_power='auto', n_components=10,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('classifier',
                 SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [107]:
y_pred = pipe.predict(X_test)

In [108]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96     47570
           1       0.84      0.66      0.74      9043

    accuracy                           0.93     56613
   macro avg       0.89      0.82      0.85     56613
weighted avg       0.92      0.93      0.92     56613



In [109]:
joblib.dump(pipe, 'model_Pipeline.pkl')
print("Pipeline Model dumped!")

# Load the model that is just saved
pipe = joblib.load('model_Pipeline.pkl')

Pipeline Model dumped!


### Using Ensemble Forest

In [110]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

# Keeping Default ,Bagging 
clf_dtc = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
scores = cross_val_score(clf_dtc, X_train, y_train, cv=5)
print(scores.mean())


clf_rfc = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
scores = cross_val_score(clf_rfc, X_train, y_train, cv=5)
print(scores.mean())


clf_etc = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
scores = cross_val_score(clf_etc, X_train, y_train, cv=5)

print(scores.mean())

0.9816472347797263
0.98057415557589
0.9742769880988964


### Selecting Top 10 features

In [111]:
%%time
X_10 = SelectKBest(chi2, k=10).fit_transform(df.iloc[:,2:59], df.iloc[:,1])
X_10.shape
X_10

CPU times: user 141 ms, sys: 135 ms, total: 275 ms
Wall time: 310 ms


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 2, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 1, 0],
       [0, 0, 3, ..., 2, 0, 0]])

In [112]:
# Splitting the data
x_train, x_test, y_train, y_test = train_test_split(
   X_10, df.iloc[:,1], test_size=0.2, random_state=42)

#### Using SVM

In [113]:
%%time
clf = SVC(gamma='auto')
clf.fit(x_train, y_train)

CPU times: user 7min 34s, sys: 1.22 s, total: 7min 35s
Wall time: 7min 36s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [114]:
y_pred = clf.predict(x_test)

In [115]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95     47570
           1       0.81      0.67      0.73      9043

    accuracy                           0.92     56613
   macro avg       0.87      0.82      0.84     56613
weighted avg       0.92      0.92      0.92     56613



In [116]:
#joblib.dump(clf, 'model_svm_10.pkl')
#print("Pipeline Model dumped!")

## Load the model that is just saved
#clf = joblib.load('model_svm_10.pkl')

## Let's see what happens if we do Dimension Reduction

Using Principal Componenet Anaylsis

In [117]:
%%time
pca = PCA(n_components=10) # Selecting 10 components
pca.fit(X_train)

CPU times: user 4.87 s, sys: 1.03 s, total: 5.9 s
Wall time: 1.1 s


PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [118]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.67234856 0.12269519 0.08465786 0.02449308 0.01218416 0.01143556
 0.01028934 0.00763465 0.00700807 0.00578607]
[4292.19571889 1833.56387412 1523.0561556   819.22650691  577.80330911
  559.7718888   530.97739291  457.37963138  438.20909516  398.17528072]


In [119]:
%%time
pca= PCA(n_components=7) # Selecting 7 components
pca.fit(X_train)

CPU times: user 3.99 s, sys: 526 ms, total: 4.51 s
Wall time: 711 ms


PCA(copy=True, iterated_power='auto', n_components=7, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [120]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.67234856 0.12269519 0.08465786 0.02449308 0.01218416 0.01143556
 0.01028933]
[4292.19571889 1833.56387412 1523.0561556   819.22650685  577.80330459
  559.77183918  530.97720335]


In [121]:
x_decom = pca.fit_transform(X_train, y=None)

Appyling to SVM

In [122]:
%%time
# SVM
clf_decom_svm = SVC(gamma='auto')
clf_decom_svm.fit(x_decom, y_train)

CPU times: user 7min 20s, sys: 2.14 s, total: 7min 22s
Wall time: 7min 22s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [123]:
X_decom_test = pca.fit_transform( X_test, y=None)
y_pred = clf_decom_svm.predict(X_decom_test)

In [124]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94     47570
           1       0.79      0.54      0.64      9043

    accuracy                           0.90     56613
   macro avg       0.85      0.76      0.79     56613
weighted avg       0.90      0.90      0.90     56613



Applying to Logistic Regression

In [125]:
%%time
# LR
results = sm.OLS(y_train,x_decom).fit() 
results.summary() 

CPU times: user 449 ms, sys: 88.6 ms, total: 538 ms
Wall time: 188 ms


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             EvalResult   R-squared (uncentered):                   0.226
Model:                            OLS   Adj. R-squared (uncentered):              0.226
Method:                 Least Squares   F-statistic:                              9438.
Date:                Tue, 26 Jan 2021   Prob (F-statistic):                        0.00
Time:                        13:18:16   Log-Likelihood:                         -83161.
No. Observations:              226451   AIC:                                  1.663e+05
Df Residuals:                  226444   BIC:                                  1.664e+05
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.322e-05   8.14e-05      0.162      0.871      -0.000       0.000
x2             0.0007      0.000      3.821      0.000       0.000       0.001
x3         -5.074e-05      0.000     -0.221      0.825      -0.001       0.000
x4             0.0632      0.000    148.259      0.000       0.062       0.064
x5             0.1222      0.001    202.180      0.000       0.121       0.123
x6            -0.0288      0.001    -46.193      0.000      -0.030      -0.028
x7            -0.0214      0.001    -32.561      0.000      -0.023      -0.020
==============================================================================
Omnibus:                    40151.175   Durbin-Watson:                   1.595
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            67458.876
Skew:                           1.186   Prob(JB):                         0.00
Kurtosis:                       4.234   Cond. No.                         8.08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [126]:
%%time
clf_lr = LogisticRegression(random_state=0).fit(x_decom, y_train)

/Users/piyushkurkure/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 811 ms, sys: 98.1 ms, total: 909 ms
Wall time: 376 ms


In [127]:
y_pred = clf_lr.predict(X_decom_test)

In [128]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92     47570
           1       0.66      0.33      0.44      9043

    accuracy                           0.87     56613
   macro avg       0.77      0.65      0.68     56613
weighted avg       0.85      0.87      0.85     56613



Applying to Random Forest

In [129]:
%%time
# RF
clf_rf = RandomForestClassifier(max_depth=10, random_state=0)
clf_rf.fit(x_decom, y_train)

/Users/piyushkurkure/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 3 s, sys: 65.5 ms, total: 3.07 s
Wall time: 2.72 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [130]:
y_pred = clf_rf.predict(X_decom_test)
accuracy_score(y_test, y_pred)

0.9177927331178352

In [131]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95     47570
           1       0.79      0.66      0.72      9043

    accuracy                           0.92     56613
   macro avg       0.86      0.82      0.84     56613
weighted avg       0.91      0.92      0.91     56613



### Answers
#### Machine Learning model used - 

A classifier that predicts the ‘EvalResult’ with good results is SVM and Random Forest.

#### Evaluation metrices used - 

Evaluation metrices used is Precision, Recall and F1-score
The greater the F1 Score, the better is the performance of our model. It basically tries to find the balance between precision and recall.

Here, precision provides number of correct positive results divided by the number of positive results predicted by our classifier. Whereas, Recall provides number of correct positive results divided by the number of all relevant samples.
At last, F1 Score is the Harmonic Mean between precision and recall. The range for F1 Score is [0,1]. It tells us how precise our classifier is i.e. how many instances it classifies correctly.



It is found that maximum values of these metrices is found when I used SVM algorithm on full data.

## Optional

#### 1. Implement a REST Web-Service wrapper for the classifier.

I have created a app.ipynb file in this folder for implementing this functionality.

I tried to implement the REST API for this classifier by generating pickles for our models using joblib library.
The model will be running on the localhost.

My directory structure is:
- app.ipynb
- classifier.ipynb

I know it is optional, but I gave it a try and learned a lot of things about wrapping a classifier due to this. 
Thanks a lot.

Instructions for implementing REST Web-Service wrapper:-

1. Navigate to the root directory where app.ipynb is running on the localhost.<br>
   Open Terminal and type ipython app.ipynb<br>
   Or<br>
   Simply run all cells in app.ipynb<br>
   
2. Open Postman
3. Change the method to POST
4. Enter localhost:5000/predict as the URL
5. Inside the Body tab choose JSON
6. Enter testing data for prediction like -
{
    "answers": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 2, 20, 1, 0]
}<br>
This is your input for all the questions present from 6054 to 8450 

7. Hit Send

You will get answer like - 
{
    "EvalResult": "1"
}

which is your required output.

#### 2. Would dimensionality reduction help get a better accuracy for the model?

Accuracy of model after the dimension reduction depends of the strucure of the data and how many dimensions we are going to reduce.
Above, I have tired dimesionality reduction using PCA anaysis upto 7 and 10 components and I also just gave try to apply our model to only top 10 features of data.

According to my observations, the precision, recall and F1 score for classifying result as 0 remains nearly same for full data and reduced data, but precision, recall and F1 score for classifying result as 1 changes if we reduce the data given.

For example,\
Consider SVM model,
  
for result = 0,\
Full Data: Precision = 0.96, Recall = 0.98, F1 Score = 0.97\
Reduced Data: Precision = 0.92, Recall = 0.97, F1 Score = 0.94
  
for result = 1,\
Full Data: Precision = 0.89, Recall = 0.80, F1 Score = 0.84\
Reduced Data: Precision = 0.78, Recall = 0.54, F1 Score = 0.64

For this data, I think data reduction does cause change in the accuracy but to a little extent. According to the performance metrices, full data gives better accuracy. This may be because quantity of EvalResults=0 is more than EvalResults=1 , thus changes the variance too.

The accuracy provided by Random Forest is also very good as compared to provided by SVM.